If JSON parsing errors in generated_answers.json

In [ ]:
# if issues in processing

import os
from datetime import datetime
from src.preprocessing.guidelines import EntityGuidelines
from src.utils.general import write_metadata_file
from src.utils.json import load_json, save_json
from src.renal_biopsy.preprocessor import RenalBiopsyProcessor
from src.renal_biopsy.qa import RenalBiopsyOllamaQA

results_dir = "src/renal_biopsy/data/runs/20241220_185154" 
model_path = "phi3.5:3.8b-mini-instruct-q8"
n_prototype = 1
n_shots = 2
include_guidelines = True

args = {'root_dir': 'src/renal_biopsy', 'model_name': model_path, 'n_shots': n_shots, 'n_prototype': n_prototype, 'include_guidelines': include_guidelines}

eg = EntityGuidelines(f'{args['root_dir']}/data/guidelines.xlsx')
processor = RenalBiopsyProcessor(guidelines=eg)
input_json = processor.create_input_json(
    data_path=f"{args['root_dir']}/data/full_data.xlsx",
    save_path=f"{args['root_dir']}/data/real_input.json",
    full=True
)
annotated_json = load_json(f'{args['root_dir']}/data/output_report_first100.json')

model = RenalBiopsyOllamaQA(model_path=model_path, root_dir="src/renal_biopsy")
generated_answers_json = load_json(f"{results_dir}/generated_answers.json") # generated_answers_modified.json"
predicted_json = model.convert_generated_answers_to_json(generated_answers=generated_answers_json, input_json=input_json, n_prototype=args['n_prototype'])

# save predicted JSON
predicted_json_path = os.path.join(results_dir, "predicted_test.json")
save_json(predicted_json, predicted_json_path)
print(f"Predicted JSON saved to {predicted_json_path}")

# Prepare metadata storage
metadata = {
    "args": args,
    "annotation_start_time": None,
    "annotation_end_time": None,
    "evaluation_start_time": None,
    "evaluation_end_time": None,
    "score_per_report": None,
    "final_score": None
}

# Evaluate model
metadata["evaluation_start_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
all_scores, score_per_report, final_score = model.evaluate(annotated_json, predicted_json, n_prototypes=args['n_prototype'])
metadata["evaluation_end_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Save evaluation results
scores_path = os.path.join(results_dir, "evaluation_scores_test.json")
save_json(all_scores, scores_path)
print(f"Evaluation results saved to {scores_path}")


# Save final scores to metadata
metadata["score_per_report"] = score_per_report
metadata["final_score"] = final_score
# Save metadata to a text file
metadata_path = os.path.join(results_dir, "metadata_test.txt")
write_metadata_file(metadata_path, metadata)
print(f"Metadata saved to {metadata_path}")

# time: 33 mins
# errors = 1